In [38]:
import cv2   
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

In [39]:
!pip install opencv-python   

In [40]:
mixer.init()
sound = mixer.Sound(r"D:\Drowsiness detection\Drowsiness detection\alarm.wav")   # used for playing sound effects, music

In [41]:
face = cv2.CascadeClassifier('haar cascade file\haarcascade_frontalface_alt.xml')   # Haar Cascade Classifier for detecting frontal faces
leye = cv2.CascadeClassifier('haar cascade file\haarcascade_lefteye_2splits.xml')   # for left eye
reye = cv2.CascadeClassifier('haar cascade file\haarcascade_righteye_2splits.xml')  # for right eye

In [42]:
lbl=['Close','Open']   # define class labels 

model = load_model('models/cnnCat2.h5')   # loading a model which we have trained
path = os.getcwd()   # retrives the current working directory
cap = cv2.VideoCapture(0)   # used to read frames from the camera
font = cv2.FONT_HERSHEY_COMPLEX_SMALL   # font of text for videoframe
count=0   # has count of number of frames
score=0   # has count of scores
thicc=2   # for thickness 
rpred=[99]   # store prediction for right eye
lpred=[99]   # store prediction for left eye


In [43]:
while(True):   # loop will go for infinite time
    ret, frame = cap.read()   # cap.read() - method to read video frame / "ret" will give us boolean value
    height,width = frame.shape[:2]   # extracting height and width of captured frame

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   # cv-2 - OpenCV library for computer vision 
    
    # It is a color conversion code indicating the conversion from BGR to grayscale.
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))   # method of the face detection classifier
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )   # creating a rectangle frame

    for (x,y,w,h) in faces:   # shows co-ordinates of faces
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
        # x,y - top-left corner of rectangle
        # x+w, y+h - bottom right of rectangle
        # (100,100,100) - showing equal intensity for RGB
        # 1 - thin border

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)   # right eye region to have dimensions (24, 24, 1), 
        # preparing it for input to a neural network.
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = np.argmax(model.predict(r_eye),axis=-1)
        if(rpred[0]==1):   # checks the predicted output and assigns a result accordingly
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break
    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = np.argmax(model.predict(l_eye),axis=-1)
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break
    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            sound.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'